#  Loading in sheets

In [1]:
import numpy as np
import pandas as pd
import TIRCP_functions
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.2f}".format

In [2]:
#Allocation PPNO Crosswalk
FILE_NAME3 = "Allocation_PPNO_Crosswalk.csv"
allocation_ppno_crosswalk = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME3}")
    
#Allocation PPNO Crosswalk
FILE_NAME4 = "Projects_PPNO.xlsx"
project_ppno_crosswalk = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME4}")


In [3]:
project_script = TIRCP_functions.project()
project_script.PPNO.unique()

array(['CP005', 'CP012', 'CP015', 'CP007', 'CP013', 'CP004', 'CP001',
       'CP003', 'CP008', 'CP006', 'CP014', 'CP011', 'CP002', 'CP010',
       'CP019', 'CP023', 'CP076', 'CP016', 'CP058', 'CP020', 'CP021',
       'CP017', 'CP022', 'CP080', '1230', 'CP025', 'CP057', 'CP026',
       '2320B', 'CP027', 'CP028', 'CP055', 'CP036', 'CP079', 'CP029',
       'CP054', 'CP051', 'CP030', 'CP031', 'CP039', 'CP053', 'CP034',
       'CP077', 'CP032', 'CP035', 'CP078', 'CP048', 'CP047', 'CP045',
       'CP046', 'CP041', 'CP033', '1155', 'CP059', 'CP060', 'CP061',
       'CP062', 'CP063', 'CP064', 'CP065', 'CP066', 'CP067', 'CP068',
       'CP069', 'CP070', 'CP071', 'CP072', 'CP073', 'CP074', 'CP075'],
      dtype=object)

In [5]:
project_test = TIRCP_functions.project()

In [11]:
PPNO_project = set(project_test.PPNO.unique().tolist())
PPNO_allocation = set(alloc_test.PPNO.unique().tolist())

In [12]:
alloc_test.loc[alloc_test['PPNO'] == '1230'] 

,Award_Year,Project_#,Award_Recipient,Implementing_Agency,Project_ID,EA,Components,Phase,Allocation_Amt_Allocation_Sheet,Expended_Amt_Allocation_Sheet,SB1_Funding,SB1_Budget_Year,GGRF_Funding,GGRF_Budget_Year,CTC_Financial_Resolution,CTC_Allocation_Amendment,CTC_Waiver,CTC_CalSTA_Waiver,PSA_#,CT_Document_#,Date_Branch_Chief_Receives_PSA,Date_Regional_Coordinator_Receives_PSA,Date_OC_Receives_PSA,Date_OPM_Receives_PSA,Date_Legal_Receives_PSA,Date_Returned_to_PM,Date_PSA_Sent_to_Local_Agency,Date_PSA_Approved_by_Local_Agency,Date_Signed_by_DRMT,PSA_Expiry_Date,LONP,Prior_Fiscal_Years_to_2020,Fiscal_Year_2020-2021,Fiscal_Year_2021-2022,Fiscal_Year_2022-2023,Fiscal_Year_2023-2024,Fiscal_Year_2024-2025,Fiscal_Year_2025-2026,Fiscal_Year_2026-2027,Fiscal_Year_2027-2028,Fiscal_Year_2028-2029,Fiscal_Year_2029-2030,Allocation_Comments,PSA_Comments,PPNO,Allocation_Date,Third_Party_Award_Date,Completion_Date,LED
51,2016,11.00,San Bernardino County Transportation Authority...,San Bernardino County Transportation Authority...,19000086,R391GD,Construct new Rail System linking San Bernardi...,CONST,9204000.00,9204000.00,0.00,2017-18,9204000.00,NaN,TIRCP-1819-02,NaN,NaN,NaN,08SBCTAPS-01\n1/14/2020,08SBCTAPS-01,NaT,NaT,NaN,NaN,NaT,NaN,NaN,NaT,2020-01-14 00:00:00,NaT,NaN,9204000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,1230,2018-10-18,2019-05-15,2022-07-31,2022-06-01


In [13]:
project_test.loc[project_test['PPNO'] == '1230'] 

,Award_Year,Project_#,Local_Agency,Vendor_ID_#,Project_Title,District,County,Key_Project_Elements,Master_Agreement_Number,Master_Agreement_Expiration_Date,Project_Manager,Regional_Coordinator,Technical_Assistance-CALTP_(Y/N),Technical_Assistance-Fleet_(Y/N),Technical_Assistance-Network_Integration_(Y/N),Technical_Assistance-Priority_Population_(Y/N),Total_Project_Cost,TIRCP_project_sheet,Allocated_Amount,Unallocated_amt_project_sheet,Percentge_Allocated,Expended_Amt_project_sheet,Other_Funds_Involved,Award_Cycle,Local_Agency_Address,Local_Agency_City,Local_Agency_Zip,Local_Agency_Contact,Local_Agency_Email,Local_Agency_Phone_Number,Comments/Additional_Contacts,PPNO
24,2016,11,San Bernardino County Transportation Authority...,TBD,Redlands Passenger Rail Project,8,SBD,Construct new Rail System linking San Bernardi...,64SBCTAMA,2024-04-01,Daniela,Ryan,No,No,No,No,265282451.00,9204000.00,9204000,0,1,9204000.00,0.00,2,"1170 W. 3rd Street, 2nd Floor",San Bernadino,92410,"Vanessa Schoenewald, Management Analyst III",vschoenewald@gosbcta.com,(909) 884-8276 x156,Primary contact,1230


In [16]:
PPNO_project - PPNO_allocation #checking for differences

set()